# Model Training Example


In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from google.colab import drive
drive.mount('/content/drive')
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

os.chdir('/content/drive/MyDrive/Colab Notebooks/')
print('Current Working Directory ' , os.getcwd())

Mounted at /content/drive
Current Working Directory  /content/drive/MyDrive/Colab Notebooks


In [ ]:
# Load the training data set
train_data = pd.read_csv('train.csv')
train_data.drop(['url_legal', 'license', 'standard_error'], axis=1, inplace=True)
train_data.head(3)

,id,excerpt,target
0,c12129c31,When the young people returned to the ballroom...,-0.340259
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118


In [ ]:
# define X, y and test
X_train = train_data.excerpt
Y_train = train_data.target

In [ ]:
# Split the training dataset to train/validation set
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)
x_train = np.array(x_train)
x_val = np.array(x_val)
y_train = np.array(y_train) 
y_val = np.array(y_val) 
X_full = np.array(X_train)
Y_full = np.array(Y_train) 

In [ ]:
# Vectorize the input database
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english',max_features=5,strip_accents='unicode')
x_train_vector = vectorizer.fit_transform (x_train).toarray()
x_val_vector = vectorizer.fit_transform (x_val).toarray() 
x_full = vectorizer.fit_transform (X_full).toarray() 

In [ ]:
# Build a simple model
def create_model():
    
    model = keras.Sequential([
        layers.Dense(units=512, activation='relu', input_shape=[5]),
        layers.Dropout(0.25),
        layers.Dense(units=256, activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(units=128, activation='relu'),
        layers.Dropout(0.25),
        # the linear output layer 
        layers.Dense(units=1, kernel_initializer='normal', activation='linear'),
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='mean_squared_error')
    
    return model

In [ ]:
# Create a KerasClassifier with best parameters
model_KR = KerasRegressor(build_fn = create_model, batch_size = 32, epochs = 50)

# Calculate the accuracy score for each fold
kfolds = cross_val_score(model_KR, x_full, Y_full, cv = 10)

#get the accuracy
print('The mean accuracy:', kfolds.mean())

Epoch 1/50
80/80 [==============================] - 1s 4ms/step - loss: 1.3641
Epoch 2/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0217
Epoch 3/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0208
Epoch 4/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0173
Epoch 5/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0175
Epoch 6/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0192
Epoch 7/50
80/80 [==============================] - 0s 3ms/step - loss: 1.0084
Epoch 8/50
80/80 [==============================] - 0s 4ms/step - loss: 1.0051
Epoch 9/50
80/80 [==============================] - 0s 3ms/step - loss: 1.0056
Epoch 10/50
80/80 [==============================] - 0s 5ms/step - loss: 1.0081
Epoch 11/50
80/80 [==============================] - 0s 3ms/step - loss: 1.0145
Epoch 12/50
80/80 [==============================] - 0s 3ms/step - loss: 1.0102
Epoch 13/50
80/80 [==============================

In [ ]:
#use callbacks
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("", monitor="val_loss", verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto', restore_best_weights=True)

In [ ]:
history = model_KR.fit(
    x_full, Y_full,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks = [early_stop, checkpoint, reduce_lr]
)

Epoch 1/50
71/71 [==============================] - 1s 6ms/step - loss: 1.4871 - val_loss: 0.7827

Epoch 00001: val_loss improved from inf to 0.78275, saving model to 
INFO:tensorflow:Assets written to: assets
Epoch 2/50
71/71 [==============================] - 0s 4ms/step - loss: 1.0406 - val_loss: 0.8081

Epoch 00002: val_loss did not improve from 0.78275
Epoch 3/50
71/71 [==============================] - 0s 4ms/step - loss: 1.0260 - val_loss: 0.8043

Epoch 00003: val_loss did not improve from 0.78275
Epoch 4/50
71/71 [==============================] - 0s 4ms/step - loss: 1.0130 - val_loss: 0.8163

Epoch 00004: val_loss did not improve from 0.78275
Epoch 5/50
71/71 [==============================] - 0s 5ms/step - loss: 1.0074 - val_loss: 0.8052

Epoch 00005: val_loss did not improve from 0.78275
Epoch 6/50
71/71 [==============================] - 0s 4ms/step - loss: 1.0027 - val_loss: 0.8206

Epoch 00006: val_loss did not improve from 0.78275

Epoch 00006: ReduceLROnPlateau reducing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english',max_features=5, strip_accents='unicode')
a = 'you are have received my hurried letter; I wish this may be more intelligible, but though not confined for time, my head is so bewildered that I cannot answer for being coherent. Dearest Lizzy, I hardly know what I would write, but I have bad news for you, and it cannot be delayed. Imprudent as the marriage between Mr. Wickham and our poor Lydia would be, we are now anxious to be assured it has taken place, for there is but too much reason to fear they are not gone to Scotland. Colonel Forster came yesterday, having left Brighton the day before, not many hours after the express. Though Lydia s short letter to Mrs. F. gave them to understand that they were going to Gretna Green, something was dropped by Denny expressing his belief that W. never intended to go there, or to marry Lydia at all, which was repeated to Colonel F., who, instantly taking the alarm, set off from B. intending to trace their route. He did trace them easily to Clapham, but no further; for on entering that place, they removed into a hackney coach, and dismissed the chaise that brought them from Epsom. All that is known after this is, that they were seen to continue the London road. I know not what to think.'
print(a.count(' '))
print(len(a))
final_test = np.array([a], dtype='object')
vectorizer.fit_transform(final_test).toarray()
final_test_vector = vectorizer.transform(final_test).toarray()
pred_test = model_KR.predict(final_test_vector)
print(final_test_vector.shape)
#pred_test_list = [i for i in pred_test]
pred_test

219
1195
(1, 5)


array(-0.55427766, dtype=float32)

In [ ]:
jsonModel = model_KR.model.to_json()
jsonModelFile = open('model.json','w')
jsonModelFile.write(jsonModel)
jsonModelFile.close()

# Model Deployment via Jupyter Notebook

In [ ]:
!pip install azureml-core

In [11]:
# Load workspace
from azureml.core import Workspace
ws = Workspace.from_config(path = "config.json")
print(ws)

Workspace.create(name='ProjectMSA-ML', subscription_id='1798214b-d783-424a-b86d-d7b26e3655a3', resource_group='ProjectMSA')


In [12]:
from azureml.core.model import Model

# Register model
model = Model.register(ws, model_name = "projectmsa", model_path = "model.json")

Registering model projectmsa


In [13]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

# Create environment
# env = Environment.from_existing_conda_environment(name = "iris-xgboost",
                                                # conda_environment_name = "azure")

env = Environment(name = "projectmsa")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score.py",
)

In [14]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(
    ws,
    "projectmsa",
    [model],
    dummy_inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-19 04:03:00+00:00 Creating Container Registry if not exists.
2021-09-19 04:03:00+00:00 Registering the environment.
2021-09-19 04:03:04+00:00 Use the existing image.
2021-09-19 04:03:04+00:00 Generating deployment configuration.
2021-09-19 04:03:05+00:00 Submitting deployment to compute..
2021-09-19 04:03:09+00:00 Checking the status of deployment projectmsa..
2021-09-19 04:07:11+00:00 Checking the status of inference endpoint projectmsa.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-09-19T04:06:15,154571500+00:00 - gunicorn/run 
2021-09-19T04:06:15,155234400+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-19T04:06:15,161896600+00:00 - rsyslog/run 
2021-09-19T04:06:15,195398200+00:00 - nginx/run 
EdgeHubConne

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("<API Endpoint Here>")
headers = {"Content-Type": "application/json"}
data = {
    "data": [[6.1, 2.8, 4.7, 1.2]],
}
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.json())

['versicolor']
